In [180]:
# importing the libraries
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import regex as re
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

In [206]:
path=r'C:\Users\pkulkarni\OneDrive - Brunswick Corporation\Desktop\Python workflow\NADA pricing'

In [181]:
options=webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver=webdriver.Chrome(options=options)

def give_input(xpath: str, value: int | str):
    driver.find_element("xpath", xpath).clear()
    driver.find_element("xpath", xpath).send_keys(value)
    time.sleep(3)

def click_item(xpath: str):
    driver.find_element('xpath', xpath).click()

def find_price_id(url):
    return re.findall('[0-9]+', str(url))[0]

### Scraping Power Boat information from NADA

In [379]:
options=webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver=webdriver.Chrome(options=options)
# get url
driver.get(url='https://www.jdpowerconnect.com')
driver.maximize_window()
# login info
give_input('/html/body/div[1]/div/div/div[2]/div/div/form/div[2]/div/input', 'max.kimmerle@boateka.com')
give_input('/html/body/div[1]/div/div/div[2]/div/div/form/div[3]/div[1]/input', 'Boateka1$2023')
# click on log in 
click_item('//*[@id="content-body"]/div/div/form/div[5]/div[1]/button')
# click on boats icon
click_item('//*[@id="navbar"]/ul/li[2]/a')
# select Power Boats
click_item('/html/body/div[1]/div/div/div[2]/form/div[2]/div/div[1]')

# iterate over the manufacturer list
df=pd.read_csv(r"C:\Users\pkulkarni\OneDrive - Brunswick Corporation\Desktop\Python workflow\NADA pricing\FMS Master Mapping.csv")
manufacturer_list=df['Manufacturer'].tolist()

In [193]:
url_list=[]

In [194]:
counter=0
for manufacturer in tqdm(manufacturer_list):
    give_input('/html/body/div[1]/div/div/div[2]/form/div[6]/div[1]/input', str(manufacturer))
    click_item('/html/body/div[1]/div/div/div[2]/div[2]/div/div/div[1]/a')
    #Select the year
    for year in range(1,12):
        try:
            click_item('/html/body/div[1]/div/div/div[2]/form/div[1]/button'+str([year]))
            time.sleep(2)
            urls= driver.find_elements("xpath", '/html/body/div[1]/div/div/div[2]/div[4]/div/table/tbody/tr[*]/td[1]/a')
            for url in urls:
                counter+=1
                dict_obj={}
                dict_obj['ID']=counter
                dict_obj['Model']=str(url.text)
                dict_obj['manufacturer']=manufacturer
                dict_obj['Year']=driver.find_element("xpath", '/html/body/div[1]/div/div/div[2]/form/div[1]/button'+str([year])).text
                dict_obj['Price_id']=str(url.get_attribute("href"))
                url_list.append(dict_obj)
        except:
            continue
    click_item('/html/body/div[1]/div/div/div[2]/ol/li[2]/a')

100%|██████████| 41/41 [28:24<00:00, 41.58s/it]


In [244]:
len(url_list)

21218

In [245]:
df_NADA_Price_ID=pd.DataFrame.from_dict(url_list, orient='columns')
df_NADA_Price_ID.set_index('ID', drop=True, inplace=True)
df_NADA_Price_ID['URL links']=df_NADA_Price_ID['Price_id']
df_NADA_Price_ID=df_NADA_Price_ID.drop(columns=['Price_id'])
df_NADA_Price_ID['Price ID']=df_NADA_Price_ID['URL links'].apply(find_price_id)
df_NADA_Price_ID['Unique ID']=df_NADA_Price_ID['Year']+str('-')+df_NADA_Price_ID['manufacturer']+str('-')+df_NADA_Price_ID['Model']
df_NADA_Price_ID.head(5)

,Model,manufacturer,Year,URL links,Price ID,Unique ID
ID,,,,,,
1,21 SSI SKI & FISH(**),Chaparral,2024,https://www.jdpowerconnect.com/Boats/32151814/...,32151814,2024-Chaparral-21 SSI SKI & FISH(**)
2,21 SSI SKI & FISH(***),Chaparral,2024,https://www.jdpowerconnect.com/Boats/32151813/...,32151813,2024-Chaparral-21 SSI SKI & FISH(***)
3,21 SSI(**),Chaparral,2024,https://www.jdpowerconnect.com/Boats/32151815/...,32151815,2024-Chaparral-21 SSI(**)
4,21 SURF(**),Chaparral,2024,https://www.jdpowerconnect.com/Boats/32151839/...,32151839,2024-Chaparral-21 SURF(**)
5,21 SSI(***),Chaparral,2024,https://www.jdpowerconnect.com/Boats/32151816/...,32151816,2024-Chaparral-21 SSI(***)


In [247]:
df_NADA_Price_ID.to_csv(path+'\Price_ID.csv')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\pkulkarni\AppData\Local\Temp\ipykernel_37476\2495273030.py:1: SyntaxWarning: invalid escape sequence '\P'
  df_NADA_Price_ID.to_csv(path+'\Price_ID.csv')


In [248]:
href_list=df_NADA_Price_ID['URL links'].to_list()

In [261]:
main_info=[]

In [444]:
df=pd.read_csv(path+'\Price_ID_Bayliner.csv')
href_list=df['URL links'].to_list()

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\pkulkarni\AppData\Local\Temp\ipykernel_37476\1748811587.py:1: SyntaxWarning: invalid escape sequence '\P'
  df=pd.read_csv(path+'\Price_ID_Bayliner.csv')


In [457]:
options=webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver=webdriver.Chrome(options=options)
driver.get(url='https://www.jdpowerconnect.com')
driver.maximize_window()
# login info
give_input('/html/body/div[1]/div/div/div[2]/div/div/form/div[2]/div/input', 'max.kimmerle@boateka.com')
give_input('/html/body/div[1]/div/div/div[2]/div/div/form/div[3]/div[1]/input', 'Boateka1$2023')
# click on log in 
click_item('//*[@id="content-body"]/div/div/form/div[5]/div[1]/button')
time.sleep(1)
for href in tqdm(href_list):
    driver.get(href)
    dict_obj={}
    column_header=[]
    column_info=[]
    columns=driver.find_elements("xpath", "/html/body/div[1]/div/div/div[2]/div[2]/div[2]/table/tbody/tr[*]/td[1]")
    for header in columns:
        column_header.append(header.text)
    info=driver.find_elements("xpath", "/html/body/div[1]/div/div/div[2]/div[2]/div[2]/table/tbody/tr[*]/td[2]")
    for information in info:
        column_info.append(information.text)
    for index, label in enumerate(column_header):
        dict_obj[label]=column_info[index]
    click_item('/html/body/div[1]/div/div/div[2]/form/div[*]/div/button')
    price_header=[]
    for element in range(2,5):
        price_header.append((driver.find_element("xpath", "/html/body/div[1]/div/div/div[2]/div[4]/div[1]/div/div"+str([element]))).text)
    click_item('/html/body/div[1]/div/div/div[2]/form/div/div[2]/div/input')
    time.sleep(1)                                 
    price_all=[]
    for year in range(5,14):
        monthly_price=[]
        time.sleep(1)
        try:
            click_item('/html/body/div[1]/div/div/div[2]/div[8]/div/div/div[2]/form/div/div[2]/select/option'+str([year]))
            for  month in range(1,12,2):
                price_type_info=[]
                try:
                    click_item('/html/body/div[1]/div/div/div[2]/div[8]/div/div/div[2]/form/div/div[1]/select/option'+str([month]))
                    click_item('/html/body/div[1]/div/div/div[2]/div[8]/div/div/div[3]/button[1]')
                    time.sleep(2)
                    for type_price in range(2,5):
                        try:
                            price_type_info.append((driver.find_element("xpath", "/html/body/div[1]/div/div/div[2]/div[4]/div[5]/div"+str([type_price]))).text)
                        except:
                            price_type_info.append((driver.find_element("xpath", "/html/body/div[1]/div/div/div[2]/div[4]/div[7]/div"+str([type_price]))).text)                       
                    monthly_price.append(price_type_info)
                    click_item('/html/body/div[1]/div/div/div[2]/form/div/div[2]/div/input')
                except:
                    continue
            price_all.append(monthly_price)
        except:
            continue
    dict_obj["Price column head"]=price_header
    dict_obj['Price']=price_all
    print(dict_obj)
    main_info.append(dict_obj)

  0%|          | 0/196 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div[2]/form/div[*]/div/button"}
  (Session info: chrome=118.0.5993.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A70C8EF2+54786]
	(No symbol) [0x00007FF6A7035612]
	(No symbol) [0x00007FF6A6EEA64B]
	(No symbol) [0x00007FF6A6F2B79C]
	(No symbol) [0x00007FF6A6F2B91C]
	(No symbol) [0x00007FF6A6F66D87]
	(No symbol) [0x00007FF6A6F4BEAF]
	(No symbol) [0x00007FF6A6F64D02]
	(No symbol) [0x00007FF6A6F4BC43]
	(No symbol) [0x00007FF6A6F20941]
	(No symbol) [0x00007FF6A6F21B84]
	GetHandleVerifier [0x00007FF6A7417F52+3524194]
	GetHandleVerifier [0x00007FF6A746D800+3874576]
	GetHandleVerifier [0x00007FF6A7465D7F+3843215]
	GetHandleVerifier [0x00007FF6A7165086+694166]
	(No symbol) [0x00007FF6A7040A88]
	(No symbol) [0x00007FF6A70

In [449]:
main_info=main_info[:1246]

In [443]:
with open(path+'\Price_ID_total_info.pkl', 'wb') as f:
        pickle.dump(main_info, f)

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\pkulkarni\AppData\Local\Temp\ipykernel_37476\2304629544.py:1: SyntaxWarning: invalid escape sequence '\P'
  with open(path+'\Price_ID_total_info.pkl', 'wb') as f:
